#### 제목: 알라딘 Open API를 사용하여 원하는 카테고리의 도서 목록 불러오기 

##### 목적: 알라딘 api를 사용해 원하는 카테고리에 속하는 도서들의 제목, 작가, 목차만 모아서 임베딩한 후, RAG 시스템에 활용한다.

##### 사용 프로그램
- FAISS DB
- BeautifulSoup
<br/>

##### 수집하는 책의 범위
- 국내 도서
- 전자책 ?
<br/>

##### 카테고리
- 고등학교 참고서
- 수험서 
- 수험서/자격증
- 외국어
- 중학교 참고서
- 중고딩 참고서
- 초등 참고서 
- 초등학교 참고서
- 컴퓨터/모바일
<br/>



---
<br/>

> 참고: [알라딘 OpenAPI 메뉴얼](https://docs.google.com/document/d/1mX-WxuoGs8Hy-QalhHcvuV17n50uGI2Sg_GHofgiePE/edit?tab=t.0#)

> 참고: [알라딘 모든 분야 카테고리](https://image.aladin.co.kr/img/files/aladin_Category_CID_20210927.xls)

In [ ]:
"""
검색어를 입력하면 isbn13을 얻어오는 코드
알라딘 OpenAPI 메뉴얼의 상품 검색 API 사용 

1. Request
- ttbkey: 알라딘 API 인증 키 (필수)
- Query: 검색어 (필수)
- QueryType: 검색어 종류
- SearchTarget: 검색 대상 
- start: 검색 결과 시작 페이지 
- MaxResults: 검색 결과 한 페이지 당 최대 출력 개수 
- CategoryId: 특정 분야로 검색 결과 제한 
- output: 출력 방법 

2. Response
- item: 상품 정보 
"""

from dotenv import load_dotenv
from bs4 import BeautifulSoup
import webbrowser
import requests
import json
import os

load_dotenv()

ttbkey = os.getenv("ALADIN_API_KEY")

# 검색된 도서를 담을 리스트 생성 
books = []

# 도서 검색어와 카테고리 ID 입력 
Query = "자이스토리"
CategoryId = 79143

# Query = input("검색어를 입력하세요: ")
# CategoryId = input("카테고리 ID: ")

# API URL
get_isbn_url = "http://www.aladin.co.kr/ttb/api/ItemSearch.aspx"

# Request 정의
isbn_params = {
    "ttbkey": ttbkey,
    "Query": Query,
    "QueryType": "Keyword",  # 제목&저자 검색
    "SearchTarget": "Book",
    "start": 1,
    "MaxResult": 10,
    "CategoryId": CategoryId,
    "output": "js",  # JSON 형식 
}

# GET 요청 
isbn_response = requests.get(url=get_isbn_url, params=isbn_params)

# 응답 확인 
if isbn_response.status_code == 200:
    # json으로 변환 
    json_data = json.loads(isbn_response.text[:-1])  # 마지막에 ;를 빼기 위함 
    
    # 도서 정보 추출 
    json_data_items = json_data["item"]
    if len(json_data_items) == 0:
        print("도서 정보가 없습니다.")
    else:
        for item in json_data_items:
            books.append(item)
else:
    print(f"GET 요청 실패: {isbn_response.status_code}")

print(books)

[{'title': '2026 수능대비 Xistory 자이스토리 수능 연도별 모의고사 고3 수학 (2025년)', 'link': 'http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=354581082&amp;copyPaper=1&amp;ttbkey=ttbysy27082026001&amp;start=api', 'author': '박소희 외 지음', 'pubDate': '2025-01-09', 'description': '최신 수능 5회, 최신 모의평가 8회, 최신 학력평가 16회, 수능 예시 1회를 월별로 구분해서 수록한 수험서다. 실제처럼 시간을 제한해서 풀어보고 채점하면 자신의 성적 위치를 정확히 파악할 수 있다. 최신 모의고사 우선 배치로 출제 흐름 변화를 저절로 파악할 수 있다.', 'creator': 'aladin', 'isbn': 'K232035147', 'isbn13': '9791162407011', 'itemId': 354581082, 'priceSales': 19350, 'priceStandard': 21500, 'stockStatus': '', 'mileage': 1070, 'cover': 'https://image.aladin.co.kr/product/35458/10/coversum/k232035147_1.jpg', 'categoryId': 77114, 'categoryName': '국내도서>고등학교참고서>수능모의고사/기출>수학영역', 'publisher': '수경출판사(학습)', 'customerReviewRank': 10}, {'title': 'Xistory 자이스토리 내신 핵심 기출 공통수학 2 (2025년) - 2022 개정 교육과정', 'link': 'http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=354581256&amp;copyPaper=1&amp;ttbkey=ttbysy27082026001&amp;start=api', 'author': '김덕환 외 지

In [51]:
"""
검색된 isbn13으로 책의 제목과 목차를 얻어오는 코드 
알라딘 OpenAPI 메뉴얼의 상품 검색 API 사용 

1. Request
- ttbkey: 알라딘 API 인증 키 (필수)
- ItemId: 상품을 구분짓는 유일한 값 (필수)
- ItemIdType: ItemId가 ISBN으로 입력됐는지, 알라딘 고유의 ItemId인지 선택 
- Output: 출력 방법 
- OptResult: [Toc, categoryIdList] (목차, 전체 분야)

2. Response

"""

import os
import requests
from dotenv import load_dotenv

load_dotenv()

ttbkey = os.getenv("ALADIN_API_KEY")
ItemId = ""  # 도서의 ISBN

# API URL
get_toc_url = "http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx"

# 검색된 도서의 목차들을 담을 리스트 생성  
tocs = []

for book in books:
    # Request 정의
    toc_params = {
        "ttbkey": ttbkey,
        "ItemId": ItemId+book["isbn13"],
        "ItemIdType": "ISBN13",
        "Output": "js",  # JSON 형식 
        "OptResult": ["Toc", "categoryIdList"]
    }

    # GET 요청 
    toc_response = requests.get(url=get_toc_url, params=toc_params)

    # 응답 확인 
    if toc_response.status_code == 200:
        # json으로 변환
        json_data = json.loads(toc_response.text[:-1])  # 마지막에 ;를 빼기 위함 
        
        # 목차 추출 
        json_data_item = json_data["item"][0]["bookinfo"]
        tocs.append(json_data_item)

    else:
        print(f"Error: {toc_response.status_code}")

print(tocs)

[{'subTitle': '', 'originalTitle': '', 'itemPage': 760, 'toc': '<p>제 1 회 2024학년도 3월 학평<BR>\n제 2 회 2023학년도 3월 학평<BR>\n제 3 회 2022학년도 3월 학평<BR>\n제 4 회 2021학년도 3월 학평<BR>\n제 5 회 2024학년도 5월 학평<BR>\n제 6 회 2023학년도 4월 학평<BR>\n제 7 회 2022학년도 4월 학평<BR>\n제 8 회 2021학년도 4월 학평<BR>\n제 9 회 2025 대비 6월 모평 <BR>\n제10 회 2024 대비 6월 모평<BR>\n제11 회 2023 대비 6월 모평<BR>\n제12 회 2022 대비 6월 모평<BR>\n제13 회 2024학년도 7월 학평<BR>\n제14 회 2023학년도 7월 학평<BR>\n제15 회 2022학년도 7월 학평<BR>\n제16 회 2021학년도 7월 학평<BR>\n제17 회 2025 대비 9월 모평 <BR>\n제18 회 2024 대비 9월 모평 <BR>\n제19 회 2023 대비 9월 모평 <BR>\n제20 회 2022 대비 9월 모평 <BR>\n제21 회 2024학년도 10월 학평 <BR>\n제22 회 2023학년도 10월 학평 <BR>\n제23 회 2022학년도 10월 학평 <BR>\n제24 회 2021학년도 10월 학평 <BR>\n제25 회 2025 대비 대학수학능력시험 <BR>\n제26 회 2024 대비 대학수학능력시험 <BR>\n제27 회 2023 대비 대학수학능력시험 <BR>\n제28 회 2022 대비 대학수학능력시험 <BR>\n제29 회 2022 대비 대학수학능력시험 예시 <BR>\n제30 회 2021 대비 대학수학능력시험</p>', 'letslookimg': ['https://image.aladin.co.kr/product/space.gif', 'https://image.aladin.co.kr/product/space.gif', 'https://image.aladin.co.kr/pro